In [1]:
import $file.^.sparkinit, sparkinit._
import $file.^.pathinit, pathinit._
import $file.^.cpinit, cpinit._
import ss.implicits._
import org.apache.spark.sql.functions._
import java.nio.file.Paths

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
19/12/10 14:01:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import $file.$          , sparkinit._

import $file.$         , pathinit._

import $file.$       , cpinit._

import ss.implicits._

import org.apache.spark.sql.functions._

import java.nio.file.Paths

In [2]:
import pureconfig._
import pureconfig.generic.auto._
import com.relatedsciences.opentargets.etl.pipeline.{EvidenceValidationPipeline, PipelineState, Pipeline}
import com.relatedsciences.opentargets.etl.configuration.Configuration.Config

import pureconfig._

import pureconfig.generic.auto._

import com.relatedsciences.opentargets.etl.pipeline.{EvidenceValidationPipeline, PipelineState, Pipeline}

import com.relatedsciences.opentargets.etl.configuration.Configuration.Config

In [3]:
val config = {
    val c = ConfigSource.file(TEST_RESOURCE_DIR.resolve("config/application.conf")).loadOrThrow[Config]
    c.copy(
        inputDir=REPO_DIR.resolve(c.inputDir).toString,
        outputDir=REPO_DIR.resolve(c.outputDir).toString,
        resourceDir=REPO_DIR.resolve(c.resourceDir).toString,
    )
}

config: Config = Config(
  "",
  "/home/eczech/repos/ot-scoring/src/test/resources/pipeline_test/input",
  "/home/eczech/repos/ot-scoring/src/test/resources/pipeline_test/output",
  "/home/eczech/repos/ot-scoring/src/main/resources/data",
  "WARN",
  "https://raw.githubusercontent.com/opentargets/json_schema/1.6.2/opentargets.json",
  ExternalConfig(
    https://storage.googleapis.com/open-targets-data-releases/19.11/input/mrtarget.data.19.11.yml,
    https://github.com/opentargets/data_pipeline/blob/329ff219f9510d137c7609478b05d358c9195579/mrtarget.es.yml
  ),
  Pipeline(
    PipelineScoring(
      true,
      false,
      false,
      "evidence_ese.json",
      Map(
        "sysbio" -> 0.5,
        "progeny" -> 0.5,
        "europepmc" -> 0.2,
        "expression_atlas" -> 0.2,
        "phenodigm" -> 0.2,
        "slapenrich" -> 0.5,
        "crisp" -> 1.0
      )
    ),
    PipelineEvidence("evidence_raw.json", "gene_ese.json"),
    Map(
      "time" -> PipelineDecoratorConfig(true)

In [4]:
val pipeline = new EvidenceValidationPipeline(ss, config)
import pipeline._

pipeline: EvidenceValidationPipeline = com.relatedsciences.opentargets.etl.pipeline.EvidenceValidationPipeline@3c1e5d62
import pipeline._

In [5]:
val op = Pipeline
  .Builder(config)
  .start("getEvidenceRawData", () => ss.read.textFile(config.rawEvidencePath))
  .andThen("runEvidenceSchemaValidation", runEvidenceSchemaValidation)
  .andThen("parseEvidenceData", parseEvidenceData).operation

op: com.relatedsciences.opentargets.etl.pipeline.Components.Operation[org.apache.spark.sql.package.DataFrame] = com.relatedsciences.opentargets.etl.pipeline.Decorator$DatasetSummaryDecorator$$anon$3@2401194a

In [6]:
val df = op.run(new PipelineState)

19/12/10 14:02:21 INFO Decorator$: Executing operation 'parseEvidenceData'
19/12/10 14:02:21 INFO Decorator$: Executing operation 'runEvidenceSchemaValidation'
19/12/10 14:02:21 INFO Decorator$: Executing operation 'getEvidenceRawData'
19/12/10 14:02:27 INFO Decorator$: Operation 'getEvidenceRawData' complete


count at Decorator.scala:67

6 / 6

count at Decorator.scala:67

1 / 1

...

count at Decorator.scala:67

6 / 6

count at Decorator.scala:67

1 / 1

df: org.apache.spark.sql.package.DataFrame = [access_level: string, disease: struct<biosample: struct<id: string, name: string>, id: string ... 2 more fields> ... 10 more fields]